In [ ]:
from IPython.display import display_html, display, Image

# Histograms, subplots, and distributions
## Via Matplotlib and friends


### Preamble:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import whattheplot as wtp

Hint: setting the `%matplotlib inline` magic will reset your rcParams - so set them afterwards!

In [ ]:
%matplotlib inline

In [ ]:
# Tweak the plot sizing:
plt.rcParams['figure.figsize']=(8,6)
# plt.rcParams['figure.dpi']=100

# And also tweak the font-sizing via a seaborn shortcut:
sns.set_context('talk')

### Data

In [ ]:
df = wtp.data.load_iris()
df.head()

### Profiling your data with histograms
Histograms can give you a useful first summary view of your data - do I have crazy outliers? Is my data normally distributed? But [binning can obscure or exaggerate features](https://www.answerminer.com/blog/binning-guide-ideal-histogram), so be aware.

We'll start with Pandas because it's super convenient here:

In [ ]:
features_df = df[df.columns[:4]]

In [ ]:
features_df.hist(bins=25);
plt.tight_layout() # Avoids overlapping annotations

### Subplots
What if we want a bit more fine control over the subplot layout? Maybe you'd like your axes all in a column?

#### Let's use  [subplots](https://matplotlib.org/gallery/subplots_axes_and_figures/subplots_demo.html) 
We'll start off by reproducing the example above:

In [ ]:
fig, axes = plt.subplots(ncols=1, nrows=len(features_df.columns), 
                         sharex=True,
                         figsize=(4,9))
for idx, colname in enumerate(features_df.columns):
    ax = axes[idx]
    ax.hist(features_df[colname], bins=25)
    ax.set_title(colname)
fig.tight_layout()

Next, let's compare across flower-types. (This is a bit unwieldy, we'll find a better alternative in a moment)

In [ ]:
fig, axes = plt.subplots(ncols=1, nrows=len(df.target.unique()), 
                         sharex=True,
                         figsize=(4,9))
axes_iter = iter(axes)
col_to_plot = 'sepal_length'
for target_name, subgroup_df in df.groupby(df.target):
    ax = next(axes_iter)
    ax.hist(subgroup_df[col_to_plot], bins=15)
    ax.set_xlabel(col_to_plot)
    ax.set_title(target_name)
fig.tight_layout()

Again, Seaborn has a super-convenient way to do this, *if* you can remember the magic incantation. It will also attempt to pick a suitable number of bins for you:

In [ ]:
facet_grid = sns.FacetGrid(df, row='target').map(plt.hist, 'sepal_length')

### KDEs

What if we wanted to compare the subgroup-distributions directly? We can overlay histograms, but the plots can be very hard to read, made worse by the fact that they're jagged:

In [ ]:
ax = plt.gca()
col_to_plot = 'sepal_length'
for target_name, subgroup_df in df.groupby(df.target):
    ax.hist(subgroup_df[col_to_plot], bins=15, alpha=0.5, label=target_name)
ax.set_xlabel(col_to_plot)
ax.legend()
ax.set_title('Overlaid histograms');

Instead, we can use a 
[Kernel Density Estimate](https://en.wikipedia.org/wiki/Kernel_density_estimation) 
([nice explanation here](https://mathisonian.github.io/kde/)) to get a smooth line representing the estimated density of our data.

You can do this manually via 
[SciPy](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.gaussian_kde.html) or 
[Scikit-Learn](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KernelDensity.html#sklearn.neighbors.KernelDensity), 
but Seaborn makes this snappy via 
[kdeplot](https://seaborn.pydata.org/generated/seaborn.kdeplot.html)
and [distplot](http://seaborn.pydata.org/generated/seaborn.distplot.html#seaborn.distplot).

In [ ]:
ax = plt.gca()
col_to_plot = 'sepal_length'
for target_name, subgroup_df in df.groupby(df.target):
    sns.kdeplot(subgroup_df[col_to_plot],  shade=True,             
                label=target_name
               )
ax.set_xlabel(col_to_plot)
ax.legend()
ax.set_title('Overlaid KDE plots');